Let's prepare some imports and helper function, that will allow us later to work with common problems.

In [4]:
# Python 3.x
from itertools import permutations, cycle, islice
from collections import defaultdict

def _input(day):
    "Open this day's input file."
    filename = 'data/input{}.txt'.format(day)
    return open(filename)

def mapt(fn, *args): 
    "Do a map, and make the results into a tuple."
    return tuple(map(fn, *args))

### --- Day 1: Inverse Captcha ---
The captcha requires you to review a sequence of digits (your puzzle input) and find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.


For example:

    1122 produces a sum of 3 (1 + 2) because the first digit (1) matches the second digit and the third digit (2) matches the fourth digit.
    1111 produces 4 because each digit (all 1) matches the next.
    1234 produces 0 because no digit matches the next.
    91212129 produces 9 because the only digit that matches the next one is the last digit, 9.


Initially, I attempted to look at the list as something we need to rotate and iterate through. It became increasingly difficult for corner cases e.g. to address the 91212129 (9) case because the only digit that matches the next one is the last digit, 9. I spent way too much time overthinking it, when I should have re-focused and thought about the remainder and modulo in the beginning.

In [5]:
array = '91212129'
chars = len(array)
print(array, chars)

res = []
for c, n in enumerate(array):
    # The % (modulo) operator yields the remainder from the division of the first argument by the second. 
    # So in case where `array[(c + 1) % chars]` is the last char, `array[(7+1)%8]` will yield 0 because 
    # there is no remainder, which means we've reached the end of the list. This will get us the 
    # first element in the array, `array[0]` as if starting iteration from the first one.
    print("counter: {}, number: {}, next: {}, same as prev: {}".format(
        c, n, array[(c + 1) % chars], n == array[(c + 1) % chars]))
    if n == array[(c + 1) % chars]:
        res.append(int(n))

print(sum(res))

91212129 8
counter: 0, number: 9, next: 1, same as prev: False
counter: 1, number: 1, next: 2, same as prev: False
counter: 2, number: 2, next: 1, same as prev: False
counter: 3, number: 1, next: 2, same as prev: False
counter: 4, number: 2, next: 1, same as prev: False
counter: 5, number: 1, next: 2, same as prev: False
counter: 6, number: 2, next: 9, same as prev: False
counter: 7, number: 9, next: 9, same as prev: True
9


Now let's simplify and run the above solution through the example cases and a few custom ones.

In [6]:
def inverse_captcha(array): 
    return sum(int(n) 
               for c, n in enumerate(array) 
               if n == array[(c + 1) % len(array)])

assert inverse_captcha('1122') == 3
assert inverse_captcha('1111') == 4
assert inverse_captcha('1234') == 0
assert inverse_captcha('91212129') == 9
assert inverse_captcha('08798780') == 0
assert inverse_captcha('3335553') == 19
assert inverse_captcha('3' * 10) == 30
assert inverse_captcha('1' * 10 + '7' * 130 + '5' * 25 ) == 1032

Now let's run it for for the input:

In [7]:
input_array = _input(1).read().strip()
print(inverse_captcha(input_array))

1175


#### --- Part Two ---
Now, instead of considering the next digit, it wants you to consider the digit halfway around the circular list. That is, if your list contains 10 items, only include a digit in your sum if the digit 10/2 = 5 steps forward matches it. Fortunately, your list has an even number of elements.

For example:

    1212 produces 6: the list contains 4 items, and all four digits match the digit 2 items ahead.
    1221 produces 0, because every comparison is between a 1 and a 2.
    123425 produces 4, because both 2s match each other, but no other digit has a match.
    123123 produces 12.
    12131415 produces 4.

Here, we only need to modify the `array[(c + 1) % len(array)]` part to include slicing list by half, such that it yields the remainder in the same way it did above. So the array part becomes ``array[(c + len(array) // 2) % len(array)]``

In [8]:
array = '123123'

sum(int(n) for c, n in enumerate(array) if n == array[(c + len(array) // 2) % len(array)])

12

Let's run it through the tests again. Before we'll try the function on the problem's input.

In [9]:
def inverse_captcha_2(array): 
    return sum(int(n) 
               for c, n in enumerate(array) 
               if n == array[(c + len(array) // 2) % len(array)])

assert inverse_captcha_2('1212') == 6
assert inverse_captcha_2('1221') == 0
assert inverse_captcha_2('123425') == 4
assert inverse_captcha_2('123123') == 12
assert inverse_captcha_2('123425') == 4
assert inverse_captcha_2('66' * 3 + '123' + '66' * 3) == 54

In [10]:
inverse_captcha_2(input_array)

1166

### --- Day 2: Corruption Checksum ---

The spreadsheet consists of rows of apparently-random numbers. To make sure the recovery process is on the right track, they need you to calculate the spreadsheet's checksum. For each row, determine the difference between the largest value and the smallest value; the checksum is the sum of all of these differences.

For example, given the following spreadsheet:

```
5 1 9 5
7 5 3 
2 4 6 8
```

    The first row's largest and smallest values are 9 and 1, and their difference is 8.
    The second row's largest and smallest values are 7 and 3, and their difference is 4.
    The third row's difference is 6.

In this example, the spreadsheet's checksum would be 8 + 4 + 6 = 18.


In [11]:
lines = [l.split() for l in _input('2sample').readlines()]
print(lines)

[['5', '1', '9', '5'], ['7', '5', '3'], ['2', '4', '6', '8']]


In [12]:
for line in lines:
    print("max: {}, min: {}, dif: {}, line {}".format(
        max(line), min(line), int(max(line))-int(min(line)), line))

max: 9, min: 1, dif: 8, line ['5', '1', '9', '5']
max: 7, min: 3, dif: 4, line ['7', '5', '3']
max: 8, min: 2, dif: 6, line ['2', '4', '6', '8']


In [13]:
sum(int(max(line)) - int(min(line)) for line in lines)

18

In [14]:
input_lines = [i.split() for i in _input(2).readlines()]
print(input_lines)

[['1364', '461', '1438', '1456', '818', '999', '105', '1065', '314', '99', '1353', '148', '837', '590', '404', '123'], ['204', '99', '235', '2281', '2848', '3307', '1447', '3848', '3681', '963', '3525', '525', '288', '278', '3059', '821'], ['280', '311', '100', '287', '265', '383', '204', '380', '90', '377', '398', '99', '194', '297', '399', '87'], ['7698', '2334', '7693', '218', '7344', '3887', '3423', '7287', '7700', '2447', '7412', '6147', '231', '1066', '248', '208'], ['3740', '837', '4144', '123', '155', '2494', '1706', '4150', '183', '4198', '1221', '4061', '95', '148', '3460', '550'], ['1376', '1462', '73', '968', '95', '1721', '544', '982', '829', '1868', '1683', '618', '82', '1660', '83', '1778'], ['197', '2295', '5475', '2886', '2646', '186', '5925', '237', '3034', '5897', '1477', '196', '1778', '3496', '5041', '3314'], ['179', '2949', '3197', '2745', '1341', '3128', '1580', '184', '1026', '147', '2692', '212', '2487', '2947', '3547', '1120'], ['460', '73', '52', '373', '41',

In [15]:
sum(int(max(line)) - int(min(line)) for line in input_lines)

27517

Looks like the answer is not quite right. Let's run some tests and try to investigate.

In [16]:
for line in input_lines:
    print("max: {}, min: {}, dif: {}, line {}".format(
        max(line), min(line), int(max(line))-int(min(line)), line))

max: 999, min: 105, dif: 894, line ['1364', '461', '1438', '1456', '818', '999', '105', '1065', '314', '99', '1353', '148', '837', '590', '404', '123']
max: 99, min: 1447, dif: -1348, line ['204', '99', '235', '2281', '2848', '3307', '1447', '3848', '3681', '963', '3525', '525', '288', '278', '3059', '821']
max: 99, min: 100, dif: -1, line ['280', '311', '100', '287', '265', '383', '204', '380', '90', '377', '398', '99', '194', '297', '399', '87']
max: 7700, min: 1066, dif: 6634, line ['7698', '2334', '7693', '218', '7344', '3887', '3423', '7287', '7700', '2447', '7412', '6147', '231', '1066', '248', '208']
max: 95, min: 1221, dif: -1126, line ['3740', '837', '4144', '123', '155', '2494', '1706', '4150', '183', '4198', '1221', '4061', '95', '148', '3460', '550']
max: 982, min: 1376, dif: -394, line ['1376', '1462', '73', '968', '95', '1721', '544', '982', '829', '1868', '1683', '618', '82', '1660', '83', '1778']
max: 5925, min: 1477, dif: 4448, line ['197', '2295', '5475', '2886', '264

Looks like from the first line the `max()` is wrong. It's returning `max: 999` where in line there are numbers over thousand: `['1364', '461', '1438', '1456', '818', '999', '105', '1065', '314', '99', '1353', '148', '837', '590', '404', '123']`

In [17]:
max(['1364', '461', '1438', '1456', '818', '999', '105', '1065', 
     '314', '99', '1353', '148', '837', '590', '404', '123']) 

'999'

In [18]:
# The optional key argument specifies a one-argument ordering function like that used for list.sort()
# To address the above case, we need to think of list items as numbers, 
# by providing key to max function as int will return the correct maximum result.
max(['1364', '461', '1438', '1456', '818', '999', '105', '1065', 
     '314', '99', '1353', '148', '837', '590', '404', '123'], key=int) 

'1456'

Now, let's run some tests:

In [19]:
def corruption_checksum(lines):
    return sum(int(max(line, key=int)) - int(min(line, key=int)) 
               for line in lines)

assert corruption_checksum([['5', '1', '9', '5']]) == 8
assert corruption_checksum([['1364', '999', '123']]) == 1241
assert corruption_checksum([['5', '1', '9', '5'], ['7', '5', '3'], ['2', '4', '6', '8']]) == 18
assert corruption_checksum([['204', '99', '235', '2281', '2848', '3307']]) == 3208

So we can either cast all elements of the list into numbers, or use the key in max set to int.

In [20]:
corruption_checksum(input_lines)

53460

#### --- Part Two ---
It sounds like the goal is to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. They would like you to find those numbers on each line, divide them, and add up each line's result.

For example, given the following spreadsheet:
```
5 9 2 8
9 4 7 3
3 8 6 5
```
    In the first row, the only two numbers that evenly divide are 8 and 2; the result of this division is 4.
    In the second row, the two numbers are 9 and 3; the result is 3.
    In the third row, the result is 2.

In this example, the sum of the results would be 4 + 3 + 2 = 9.

In [21]:
test_input = [['5', '9', '2', '8'], ['9', '4', '7', '3'], ['3', '8', '6', '5']]

# let's try the brute force solution first
res = []
arr = mapt(int, ['5', '9', '2', '8'])

# by creating permutations from the array
perm = [i for i in permutations(arr, 2)]
print(perm)

# and then checking if the modulo remainder is 0, will give us the pair
print([i for i in perm if i[0]%i[1]==0])

# from which we can get the two numbers that are evenly divisible between each other
print([i[0]//i[1] for i in perm if i[0]%i[1]==0])

res = []
for l in test_input:
    p = [i for i in permutations(list(map(int, l)), 2)]
    print(p)
    print([i[0]//i[1] for i in p if i[0]%i[1]==0])
    res.append([i[0]//i[1] for i in p if i[0]%i[1]==0][0])
    
print(sum(res))

[(5, 9), (5, 2), (5, 8), (9, 5), (9, 2), (9, 8), (2, 5), (2, 9), (2, 8), (8, 5), (8, 9), (8, 2)]
[(8, 2)]
[4]
[(5, 9), (5, 2), (5, 8), (9, 5), (9, 2), (9, 8), (2, 5), (2, 9), (2, 8), (8, 5), (8, 9), (8, 2)]
[4]
[(9, 4), (9, 7), (9, 3), (4, 9), (4, 7), (4, 3), (7, 9), (7, 4), (7, 3), (3, 9), (3, 4), (3, 7)]
[3]
[(3, 8), (3, 6), (3, 5), (8, 3), (8, 6), (8, 5), (6, 3), (6, 8), (6, 5), (5, 3), (5, 8), (5, 6)]
[2]
9


Let's run some tests to make sure we're not cutting corners here. Perhaps some optimisation is also due for the function.

In [22]:
def corruption_checksum_2(lines):
    return sum([i[0]//i[1] for i in permutations(list(map(int, l)), 2) if i[0]%i[1]==0][0] for l in lines)

assert corruption_checksum_2(test_input) == 9
assert corruption_checksum_2([['5', '9', '2', '8', '8']]) == 4
assert corruption_checksum_2([['5', '9', '2', '8', '4']]) == 4
assert corruption_checksum_2([['3', '1']]) == 3
assert corruption_checksum_2([['30', '10']]) == 3
assert corruption_checksum_2([['500', '10', '5']]) == 50

In [23]:
corruption_checksum_2(input_lines)

282

### --- Day 3: Spiral Memory ---

You come across an experimental new kind of memory stored on an infinite two-dimensional grid.

Each square on the grid is allocated in a spiral pattern starting at a location marked 1 and then counting up while spiraling outward. For example, the first few squares are allocated like this:
```
17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...
```
While this is very space-efficient (no squares are skipped), requested data must be carried back to square 1 (the location of the only access port for this memory system) by programs that can only move up, down, left, or right. They always take the shortest path: the Manhattan Distance between the location of the data and square 1.

For example:

    Data from square 1 is carried 0 steps, since it's at the access port.
    Data from square 12 is carried 3 steps, such as: down, left, left.
    Data from square 23 is carried only 2 steps: up twice.
    Data from square 1024 must be carried 31 steps.

How many steps are required to carry the data from the square identified in your puzzle input all the way to the access port?

Your puzzle input is `347991`.

In [24]:
N = 347991

In [25]:
# generate spiral up to N
# counting number of steps
# compute manhattan distance

def left(x,y): return x-1, y
def right(x,y): return x+1, y
def up(x,y): return x, y+1
def down(x,y): return x, y-1

# moves, are to be exactly in direction of the spiral
moves = [right, down, left, up] 

In [26]:
def spiral(end):
    _moves = cycle(moves)
    n = 1
    pos = (0, 0)
    times_to_move = 1
    yield pos

    while True:
        for _ in range(2):
            move = next(_moves)
            for _ in range(times_to_move):
                if n >= end:
                    return
                pos = move(*pos)
                n += 1
                yield pos
        times_to_move += 1

In [27]:
list(spiral(12))

[(0, 0),
 (1, 0),
 (1, -1),
 (0, -1),
 (-1, -1),
 (-1, 0),
 (-1, 1),
 (0, 1),
 (1, 1),
 (2, 1),
 (2, 0),
 (2, -1)]

In [28]:
def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

def X(point): x, y = point; return x
def Y(point): x, y = point; return y

def cityblock_distance(p, q=(0, 0)): 
    "Manhattan distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

In [29]:
# let's prove that 1024 is 31 steps, and so on
assert cityblock_distance(nth(spiral(1), 1 - 1)) == 0
assert cityblock_distance(nth(spiral(12), 12 - 1)) == 3
assert cityblock_distance(nth(spiral(23), 23 - 1)) == 2
assert cityblock_distance(nth(spiral(1024), 1024 - 1)) == 31

In [30]:
cityblock_distance(nth(spiral(N), N - 1))

480

#### --- Part Two ---

As a stress test on the system, the programs here clear the grid and then store the value 1 in square 1. Then, in the same allocation order as shown above, they store the sum of the values in all adjacent squares, including diagonals.

So, the first few squares' values are chosen as follows:

    Square 1 starts with the value 1.
    Square 2 has only one adjacent filled square (with value 1), so it also stores 1.
    Square 3 has both of the above squares as neighbors and stores the sum of their values, 2.
    Square 4 has all three of the aforementioned squares as neighbors and stores the sum of their values, 4.
    Square 5 only has the first and fourth squares as neighbors, so it gets the value 5.

Once a square is written, its value does not change. Therefore, the first few squares would receive the following values:
```
147  142  133  122   59
304    5    4    2   57
330   10    1    1   54
351   11   23   25   26
362  747  806--->   ...
```
What is the first value written that is larger than your puzzle input?

Your puzzle input is still `347991`.

In [31]:
# Looks like we'll need to get the neighbors for the spiral grid above.
# for pt in spiral(24):
#     print(pt)
    

### --- Day 4: High-Entropy Passphrases ---
A new system policy has been put in place that requires all accounts to use a passphrase instead of simply a password. A passphrase consists of a series of words (lowercase letters) separated by spaces.

To ensure security, a valid passphrase must contain no duplicate words.

For example:

    aa bb cc dd ee is valid.
    aa bb cc dd aa is not valid - the word aa appears more than once.
    aa bb cc dd aaa is valid - aa and aaa count as different words.

The system's full passphrase list is available as your puzzle input. How many passphrases are valid?

In [32]:
l = ["aa bb cc dd ee", "aa bb cc dd aa", "aa bb cc dd aaa"]

# we can use set to identify unique words in line
[set(i.split()) for i in l]

[{'aa', 'bb', 'cc', 'dd', 'ee'},
 {'aa', 'bb', 'cc', 'dd'},
 {'aa', 'aaa', 'bb', 'cc', 'dd'}]

In [33]:
# so we can compare if the length of the password is equivalent to the length of items in the set
c = []
for line in l:
    w = line.split()
    c.append(len(w) == len(set(w)))

c.count(True)

2

In [34]:
def is_password_valid(lines):
    c = []
    for line in lines:
        w = line.split()
        c.append(len(w) == len(set(w)))
    return c.count(True)

assert is_password_valid(["aa bb cc dd ee", "aa bb cc dd aa", "aa bb cc dd aaa"]) == 2

In [35]:
is_password_valid(_input(4).readlines())

477

#### --- Part Two ---

For added security, yet another system policy has been put in place. Now, a valid passphrase must contain no two words that are anagrams of each other - that is, a passphrase is invalid if any word's letters can be rearranged to form any other word in the passphrase.

For example:

    abcde fghij is a valid passphrase.
    abcde xyz ecdab is not valid - the letters from the third word can be rearranged to form the first word.
    a ab abc abd abf abj is a valid passphrase, because all letters need to be used when forming another word.
    iiii oiii ooii oooi oooo is valid.
    oiii ioii iioi iiio is not valid - any of these words can be rearranged to form any other word.

Under this new system policy, how many passphrases are valid?


In [36]:
# Here we need to check exclude the words that are anagrams of each other

l = ["abcde xyz ecdab", "iiii oiii ooii oooi oooo", "oiii ioii iioi iiio"]  # True, False
def is_password_valid_2(lines):
    c = []
    for line in lines:
        w = line.split()
        # sort letters, so that the set can exclude duplicates
        # so `"abcde xyz ecdab"` becomes `'abcde', 'xyz', 'abcde'`
        w_s = [''.join(sorted(i)) for i in w] 
        c.append(len(w_s) == len(set(w_s)))
    return c.count(True)


assert is_password_valid_2(l) == 1

In [37]:
is_password_valid_2(_input(4).readlines())

167

### --- Day 5: A Maze of Twisty Trampolines, All Alike ---

An urgent interrupt arrives from the CPU: it's trapped in a maze of jump instructions, and it would like assistance from any programs with spare cycles to help find the exit.

The message includes a list of the offsets for each jump. Jumps are relative: -1 moves to the previous instruction, and 2 skips the next one. Start at the first instruction in the list. The goal is to follow the jumps until one leads outside the list.

In addition, these instructions are a little strange; after each jump, the offset of that instruction increases by 1. So, if you come across an offset of 3, you would move three instructions forward, but change it to a 4 for the next time it is encountered.

For example, consider the following list of jump offsets:
```
0
3
0
1
-3
```
Positive jumps ("forward") move downward; negative jumps move upward. For legibility in this example, these offset values will be written all on one line, with the current instruction marked in parentheses. The following steps would be taken before an exit is found:
```
    (0) 3  0  1  -3  - before we have taken any steps.
    (1) 3  0  1  -3  - jump with offset 0 (that is, don't jump at all). Fortunately, the instruction is then incremented to 1.
     2 (3) 0  1  -3  - step forward because of the instruction we just modified. The first instruction is incremented again, now to 2.
     2  4  0  1 (-3) - jump all the way to the end; leave a 4 behind.
     2 (4) 0  1  -2  - go back to where we just were; increment -3 to -2.
     2  5  0  1  -2  - jump 4 steps forward, escaping the maze.
```
In this example, the exit is reached in 5 steps.

How many steps does it take to reach the exit?


In [38]:
def run(program):
    memory = list(program)
    pc = steps = 0
    while pc in range(len(memory)):
        steps += 1
        oldpc = pc
        pc += memory[pc]
        memory[oldpc] += 1
    return steps

run([0,3,0,1,-3])

5

In [43]:
program = mapt(int, _input(5).readlines())
program[:10]

(0, 1, 1, -2, -2, -2, -2, -4, 2, 0)

In [46]:
run(program)

373160

#### --- Part Two ---

Now, the jumps are even stranger: after each jump, if the offset was three or more, instead decrease it by 1. Otherwise, increase it by 1 as before.

Using this rule with the above example, the process now takes 10 steps, and the offset values after finding the exit are left as 2 3 2 3 -1.

How many steps does it now take to reach the exit?


In [50]:
def run_2(program):
    memory = list(program)
    pc = steps = 0
    while pc in range(len(memory)):
        steps += 1
        oldpc = pc
        pc += memory[pc]
        memory[oldpc] += (-1 if memory[oldpc] >= 3 else 1)
    return steps

run_2([0,3,0,1,-3])

10

In [52]:
run_2(program)

26395586

In [53]:
#Testing and verification

### --- Day 6: Memory Reallocation ---

A debugger program here is having an issue: it is trying to repair a memory reallocation routine, but it keeps getting stuck in an infinite loop.

In this area, there are sixteen memory banks; each memory bank can hold any number of blocks. The goal of the reallocation routine is to balance the blocks between the memory banks.

The reallocation routine operates in cycles. In each cycle, it finds the memory bank with the most blocks (ties won by the lowest-numbered memory bank) and redistributes those blocks among the banks. To do this, it removes all of the blocks from the selected bank, then moves to the next (by index) memory bank and inserts one of the blocks. It continues doing this until it runs out of blocks; if it reaches the last memory bank, it wraps around to the first one.

The debugger would like to know how many redistributions can be done before a blocks-in-banks configuration is produced that has been seen before.

For example, imagine a scenario with only four memory banks:

    The banks start with 0, 2, 7, and 0 blocks. The third bank has the most blocks, so it is chosen for redistribution.
    Starting with the next bank (the fourth bank) and then continuing to the first bank, the second bank, and so on, the 7 blocks are spread out over the memory banks. The fourth, first, and second banks get two blocks each, and the third bank gets one back. The final result looks like this: 2 4 1 2.
    Next, the second bank is chosen because it contains the most blocks (four). Because there are four memory banks, each gets one block. The result is: 3 1 2 3.
    Now, there is a tie between the first and fourth memory banks, both of which have three blocks. The first bank wins the tie, and its three blocks are distributed evenly over the other three banks, leaving it with none: 0 2 3 4.
    The fourth bank is chosen, and its four blocks are distributed such that each of the four banks receives one: 1 3 4 1.
    The third bank is chosen, and the same thing happens: 2 4 1 2.

At this point, we've reached a state we've seen before: 2 4 1 2 was already seen. The infinite loop is detected after the fifth block redistribution cycle, and so the answer in this example is 5.

Given the initial block counts in your puzzle input, how many redistribution cycles must be completed before a configuration is produced that has been seen before?
